In [ ]:
import pandas
from pathlib import Path
from tqdm import tqdm
from PIL import Image
from io import BytesIO
import json
import torch
from torchvision import transforms, models
from pytorch_lightning import LightningModule, seed_everything, Trainer
import requests
from multiprocessing import Pool
import random

In [ ]:
!mkdir images
!touch images/missing_pictures.txt
!wget https://github.com/metmuseum/openaccess/raw/master/MetObjects.csv

# Egyptian Museum
The goal of this experiment is to determine whether we can use machine learning to date Egyptian artifacts using only their photographs.

The artifacts are very varied in their appearance, material used and size. We will only focus on the appearance for now.

In [ ]:
df = pandas.read_csv('MetObjects.csv')
egypt = df[df['Department']=='Egyptian Art']
egypt

## Getting images
Here we download all possible images of the egyptian artifacts.

# !!!!! This process might be interrupted by the server. If that happens, re-run this cell. !!!!!

In [ ]:
Path("images").mkdir(exist_ok=True)
def download_image_for_id(obj_id):
    response = json.loads(requests.get(url=f'https://collectionapi.metmuseum.org/public/collection/v1/objects/{obj_id}').content)
    if not "primaryImageSmall" in response or not response['primaryImageSmall']:
        print(obj_id, file=open("images/missing_pictures.txt", "a+"))
        return
    image_response = requests.get(url=response['primaryImageSmall'])
    if image_response.status_code != 200:
        print(obj_id, file=open("images/missing_pictures.txt", "a+"))
        return
    image_bytes = image_response.content
    image = Image.open(BytesIO(image_bytes))
    image.save(f"images/{obj_id}.jpg")

all_ids = egypt['Object ID'].values
with open("images/missing_pictures.txt", "r") as f:
    known_missing_pictures = [int(line) for line in f]
missing_ids = [obj_id for obj_id in all_ids if not Path(f"images/{obj_id}.jpg").exists() and obj_id not in known_missing_pictures]
if missing_ids:
    with Pool(10) as pool:
        _ = list(tqdm(pool.imap(download_image_for_id, missing_ids), total=len(missing_ids)))

## Dataset preprocessing:
The museum contains Egyptian artifacts ranging from the Neolithic to modern era. We will only focus on the dynastic period, ~3500 BCE - 30BCE (Roman annexation of Egypt).
* All pictures will be resized to 224, 224 (as AlexNet was trained on).
* Some pictures are originally greyscale. These will be treated as colored pictures nonetheless.
* Start and end dates will be normalized (we make the big assumption that the distribution of dates is gaussian)

In [ ]:
from matplotlib import pyplot as plt
plt.hist(egypt[(egypt['Object Begin Date']>-3500) & (egypt['Object End Date']<0)]['Object Begin Date'])

In [ ]:
egypt_dynastic = egypt[(egypt['Object Begin Date']>-3500) & (egypt['Object End Date']<0)]
egypt_with_pictures = egypt_dynastic[egypt_dynastic['Object ID'].apply(lambda obj_id: Path(f'images/{obj_id}.jpg').exists())]
random.seed(147)
val_obj_ids = random.sample(list(egypt_with_pictures['Object ID'].values), int(len(egypt_with_pictures)*0.1))
train_dataset, val_dataset = [], []
class UnGreyScale(torch.nn.Module):
    def forward(self, tensor):
        if tensor.shape[0] == 3: return tensor
        elif tensor.shape[0] == 1: return tensor.repeat(3, 1, 1)
        else:
            raise NotImplementedError

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    UnGreyScale(),
])
total_epoch_mean = egypt_with_pictures[['Object Begin Date', 'Object End Date']].values.mean()
total_epoch_std = egypt_with_pictures[['Object Begin Date', 'Object End Date']].values.std()

for obj_id, min_year, max_year in tqdm(egypt_with_pictures[['Object ID', 'Object Begin Date', 'Object End Date']].values):
    image = Image.open(f'images/{obj_id}.jpg')
    image = transform(image)
    min_year_norm = (min_year - total_epoch_mean) / total_epoch_std
    max_year_norm = (max_year - total_epoch_mean) / total_epoch_std
    if obj_id in val_obj_ids:
        val_dataset.append({'inputs':image, 'min_date':min_year_norm, 'max_date':max_year_norm})
    else:
        train_dataset.append({'inputs':image, 'min_date':min_year_norm, 'max_date':max_year_norm})
seed_everything(147, workers=True)

## Model
Model is based on the AlexNet, a pre-trained computer vision model that was trained on the ImageNet dataset.

We add our own regression layer at the end, and set learning rates. We fine-tune also the AlexNet's parameters.

The loss function is the MSE, counted from the center of the time interval the artifact was dated to. Experiments with tweaked MSE were made, but those did not improve performance.

A prediction is considered accurate, if it falls in the interval the artifact was dated to.

In [ ]:
# def interval_loss(label_min, label_max, pred):
#     label_mid_dist =  (pred - (label_min + label_max) / 2) ** 2
#     lower_loss = (pred < label_min) * label_mid_dist
#     upper_loss = (pred > label_max) * label_mid_dist
#     middle_loss = ((label_max > pred) & (pred > label_min)) * label_mid_dist
#     return torch.mean(lower_loss + upper_loss + 0.2 * middle_loss)

def accuracy(label_min, label_max, pred):
    return ((label_max>=pred)&(pred>=label_min)).sum()/label_min.shape[0]

class DateClassifier(LightningModule):
    def __init__(self):
        super().__init__()
        self.model = models.alexnet(pretrained=True)
        regressor = torch.nn.Sequential(
            torch.nn.Dropout(0.5),
            torch.nn.Linear(9216, 4096),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(4096, 4096),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(4096, 1),
        )
        #self.model.requires_grad = False
        # last_ftrs = self.model.fc.in_features
        # last_layer = torch.nn.Linear(last_ftrs, 1)
        self.model.classifier = regressor

    def forward(self, batch):
        return self.model(batch)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam([
            {
                'params':[param for name, param in self.model.named_parameters() if not name.startswith("model.classifier")],
                'lr': 1e-4
            },
            {
                'params':[param for name, param in self.model.named_parameters() if name.startswith('model.classifier')],
                'lr':1e-2
             }
        ])
        return optimizer

    def training_step(self, batch, batch_idx):
        self.train()
        outs = self(batch['inputs']).squeeze()
        # loss = interval_loss(batch['min_date'], batch['max_date'], outs)
        loss = torch.nn.functional.mse_loss(outs, (batch['min_date'] + batch['max_date']).float()/2)
        acc = accuracy(batch['min_date'], batch['max_date'], outs)
        self.log('train_loss', loss)
        self.log('train_accuracy', acc)
        return loss
    def validation_step(self, batch, batch_idx):
        self.eval()
        outs = self(batch['inputs']).squeeze()
        # loss = interval_loss(batch['min_date'], batch['max_date'], outs)
        loss = torch.nn.functional.mse_loss(outs, (batch['min_date'] + batch['max_date']).float()/2)
        acc = accuracy(batch['min_date'], batch['max_date'], outs)
        self.log('val_loss',loss)
        self.log('val_accuracy',acc)
    def test_step(self, batch, batch_idx):
        self.eval()
        outs = self(batch['inputs']).squeeze()
        acc = accuracy(batch['min_date'], batch['max_date'], outs)
        self.log('test_accuracy',acc)
        return acc

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

lightning_model = DateClassifier()
trainer = Trainer(gpus=-1 if torch.cuda.is_available() else 0, precision=16, max_epochs=100)
trainer.fit(lightning_model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)
trainer.test(lightning_model, val_dataloader)

# Evaluation
Evaluation can be done using tensorboard, as in the cell below.

## Comments
My home experiments yielded a very disappointing accuracy of ~21%. This however could be most certainly improved, given enough time. A few areas of possible improvement come to mind:
* The pictures were not cropped in any way, and the backgrounds on them are vastly different from each other.
* There was no pre-processing transformation of the pictures (e.g. rotation, distortion, mirroring...)
* The dataset contains information on the materials used and the sizes of the artifacts. That could also possibly help.
* Archeologicaly very important information like the name of the site the artifact was found in was also not used.
* More experiments with tuning hyperparameters and loss functions could be made.

In [ ]:
!tensorboard --logdir ./lightning_logs